In [4]:
import json
import time
from time import sleep
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pandas import *

In [5]:
#가게 이름, 가게 유형, 주소, 대표 메뉴 1개, 메뉴 설명, 리뷰 5개
def search_food(key):
    try:
        food_list = []
        
        url = 'https://map.naver.com/v5/search'
        driver = webdriver.Chrome()  # 드라이버 경로
        # driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
        driver.get(url)
        key_word = key  # 검색어

        # css 찾을때 까지 10초대기
        def time_wait(num, code):
            try:
                wait = WebDriverWait(driver, num).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, code)))
            except:
                print(code, '태그를 찾지 못하였습니다.')
                driver.quit()
            return wait

        # frame 변경 메소드
        def switch_frame(frame):
            driver.switch_to.default_content()  # frame 초기화"
            driver.switch_to.frame(frame)  # frame 변경

        # 페이지 다운
        def page_down(num):
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            body.click()
            for i in range(num):
                body.send_keys(Keys.PAGE_DOWN)

        # css를 찾을때 까지 10초 대기
        time_wait(10, 'div.input_box > input.input_search')

        # (1) 검색창 찾기
        search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
        search.send_keys(key_word)  # 검색어 입력
        search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

        sleep(5)
        
        url_str = driver.current_url
        if url_str.find("isCorrectAnswer") == -1:
            print(url_str)
            print("결과 여러 개 존재(지점 또는 근접 지역)")
            
            search = driver.find_elements(By.ID, "searchIframe")
            # frame 변경
            switch_frame('searchIframe')
            sleep(2)
        
            
            tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.YwYLL')
            if tmp_shop == []:
                tmp_shop = driver.find_elements(By.CSS_SELECTOR, '.place_bluelink')
            tmp_shop[0].click()
            
            sleep(5)
        
        else:
            print(url_str)
            print("결과 한 개")
        
        # (2) frame 변경
        switch_frame('entryIframe')
        #page_down(40)
        sleep(2)

        # 시작시간
        start = time.time()
        print('\n[크롤링 시작...]')
        
        shop_name = []
        shop_type = []
        shop_address = []
        menu_name = []
        menu_story = []
        review_1 = []
        review_2 = []
        review_3 = []
        review_4 = []
        review_5 = []
        
        place_star = driver.find_elements(By.CSS_SELECTOR, '.place_blind')
        
        
            
        
        #가게 이름
        names = driver.find_elements(By.CSS_SELECTOR, '.GHAhO')  # (3) 장소명
        shop_name.append(names[0].text)
        print(names[0].text)
        
        #가게 타입
        types = driver.find_elements(By.CSS_SELECTOR, '.lnJFt')  # (4) 장소 유형
        shop_type.append(types[0].text)
        print(types[0].text)
        
        #가게 주소
        address = driver.find_elements(By.CSS_SELECTOR, '.LDgIH') # (5) 주소
        shop_address.append(address[0].text)
        print(address[0].text)
        
        #버튼 정의
        home_list = driver.find_elements(By.CSS_SELECTOR, ".veBoZ") # (6) 리뷰
        
        #잠시 대기
        sleep(3)
        
        
        #메뉴 버튼 클릭
        for k in range(len(home_list)):
            if home_list[k].text == "리뷰":
                #print(home_list[k].text)
                home_list[k].click()
                sleep(3)
                break
            else:
                continue
        
        
        #리뷰 찾기
        revew_love_btn = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div[2]/div[1]/h2/span')
        
        if len(revew_love_btn) != 0:
            tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            return tmp
        
        review_btn = driver.find_elements(By.CSS_SELECTOR, ".t3JSf") # (7) 리뷰
        
        #리뷰 1
        real_review = review_btn[0].text.split('"')
        review_1.append(real_review[1])
        print(real_review[1])
        #리뷰 2
        real_review = review_btn[1].text.split('"')
        review_2.append(real_review[1])
        print(real_review[1])
        #리뷰 3
        real_review = review_btn[2].text.split('"')
        review_3.append(real_review[1])
        print(real_review[1])
        #리뷰 4
        real_review = review_btn[3].text.split('"')
        review_4.append(real_review[1])
        print(real_review[1])
        #리뷰 5
        real_review = review_btn[4].text.split('"')
        review_5.append(real_review[1])
        print(real_review[1])
        
        tmp = DataFrame({'가게명': shop_name, '가게 유형' :shop_type, '가게 주소': shop_address, "리뷰1": review_1, "리뷰2": review_2, "리뷰3": review_3, "리뷰4": review_4,"리뷰5": review_5})

        print(f'{shop_name} ...완료')

        sleep(1)

        print('[데이터 수집 완료]\n소요 시간 :', time.time() - start, "\n")
        driver.quit()  # 작업이 끝나면 창을 닫는다.
        
        return tmp
    except:
         tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
         return tmp

In [ ]:

regions = ["종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구", "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구", "양천구", 
           "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구", "서초구", "강남구", "송파구", "강동구"]

#len(regions)
for i in range(len(regions)):
    tmp = DataFrame(columns = ['가게명', '가게 유형', '가게 주소', '리뷰1', '리뷰2', '리뷰3', '리뷰4', '리뷰5'])
    filename = "./out_sleep/out_" + str(i) + "_sleep.csv"
    df = read_csv(filename, header = None)
    df.columns = ['가게명']
    
    for j in range(len(df)):
        tp = df.loc[j]['가게명']
        key = "서울 " + regions[i] + " " + tp
        print(str(j) + "번째 가게")
        
        data = search_food(key)
        
        result2 = concat([tmp, data], ignore_index = True)
        tmp = result2

    #파일 저장
    filename = "./out_sleep_review/out_" + str(i) + "_sleep.csv"
    tmp.to_csv(filename, header = None, index = False)
    print(filename + "완료")


In [ ]:
#지금 번호 확인
print(i)
print(j)

In [ ]:
# 합쳐야 할 때 맨 처음 한 번만 실행
# ex = tmp.copy()

In [ ]:
ex2 = tmp.copy()
ex2

In [ ]:
import pandas as pd
tmp_df = pd.concat([ex,ex2], axis=0, ignore_index=True)
tmp_df[-3:]

In [ ]:
ex = tmp_df.copy()
ex[-3:]

In [ ]:
star_yes = 0
        
        for i in range(len(place_star)):
            if "별점" in place_star[i].text:
                star_yes = 1
                break
            else:
                star_yes = 0
        print(star_yes)
        
        #별점 없으면 종료
        if star_yes == 0:
            tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            return tmp
                
        star_text = driver.find_elements(By.CSS_SELECTOR, '.PXMot')
        #print(star_text[0].text) #별점 숫자 출력
        
        #숫자만 건지기
        real_star = str(star_text[0].text).split("\n")[1]
        print(real_star) #별점 숫자 출력
        
        #별점 4.3보다 낮으면 없애기
        if float(real_star) < 3.8:
            tmp = DataFrame({'가게명': "-", '가게 유형' :"-", '가게 주소': "-", "리뷰1": "-", "리뷰2": "-", "리뷰3": "-", "리뷰4": "-","리뷰5": "-"}, index = [0])
            return tmp